In [2]:
import pandas as pd

csv1 = pd.read_csv('../data/snyk_crawl_with_git.csv')

csv2 = pd.read_csv('../data/advisory_crawl_with_git.csv')

print(len(csv1), len(csv2))

13828 17356


In [3]:
import pandas as pd

# Load the CSV files

# Extract relevant columns
df1 = csv1[['ID', 'Package', 'CWE', 'CVE', 'Date', 'disclosed', 'Platform', 'Version Range', 'GitHub URL']]
df2 = csv2[['ID', 'PACKAGE_NAME', 'CWE_IDS', 'ALIASES', 'PUBLISH_DATE', 'ECOSYSTEM', 'VERSION_RANGE', 'GitHub URL']]

# Clean CWE column in df2 for merging
df2['CWE'] = df2['CWE_IDS'].str.replace("['", "").str.replace("']", "").str.replace("'", "").str.split(", ")
df2 = df2.explode('CWE')
df2.drop(columns=['CWE_IDS'], inplace=True)
df2['PUBLISH_DATE'] = pd.to_datetime(df2['PUBLISH_DATE']).dt.strftime('%-d %b %Y')

# Rename PUBLISH_DATE to Date for consistency
df1.rename(columns={'disclosed': 'Disclosed_Date'}, inplace=True)

df2.rename(columns={'PUBLISH_DATE': 'Date', 'ALIASES': 'CVE', 'ECOSYSTEM': 'Platform', 'PACKAGE_NAME': 'Package', 'VERSION_RANGE': 'Version Range'}, inplace=True)

# print("Unique Platforms Df1:", df1['Platform'].unique())

# print("Unique Platforms Df2:", df2['Platform'].unique())


df1['Platform'] = df1['Platform'].replace({
    'Unmanaged (C/C++)': 'C/C++',
    'pip': 'PyPI',
    'npm': 'NPM',
    'nuget': 'NuGet',
    'golang': 'Go',
    'composer': 'Composer',
    'Cargo': 'Crates',
    'cargo': 'Crates',
    'maven': 'Maven',
    'rubygems': 'Rubygems'
})

df1['Source'] = 'Snyk'

df2['Platform'] = df2['Platform'].replace({
    'Packagist': 'Composer',
    'npm': 'NPM',
    'crates.io': 'Crates',
})

df2['Source'] = 'Git'

selected_ecosystems = ['NPM', 'RubyGems', 'PyPI', 'Go', 'Maven', 'NuGet', 'C/C++', 'Composer', 'Crates']


df1 = df1[df1['Platform'].isin(selected_ecosystems)]
df1 = df1[df1['CWE'] != '[]']

df2 = df2[df2['Platform'].isin(selected_ecosystems)]
df2 = df2[df2['CWE'] != '[]']

final_df = pd.concat([df1, df2], ignore_index=True)
print(len(df1), len(df2), len(final_df))
print(set(final_df['Platform'].tolist()))
# final_df = final_df[final_df['Platform'].isin(selected_ecosystems)]
# final_df = final_df[final_df['CWE'] != '[]']

# # Merge the two DataFrames on CWE
# # combined_df = pd.merge(df1, df2, on='CWE', how='outer')

# unique_platforms = final_df['Platform'].unique()
# print("Unique Platforms:", unique_platforms)

# Display the combined DataFrame
final_df

13619 19782 33401
{'PyPI', 'Crates', 'NuGet', 'C/C++', 'RubyGems', 'Composer', 'Go', 'Maven', 'NPM'}


/tmp/ipykernel_2563741/138892696.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['CWE'] = df2['CWE_IDS'].str.replace("['", "").str.replace("']", "").str.replace("'", "").str.split(", ")
/tmp/ipykernel_2563741/138892696.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'disclosed': 'Disclosed_Date'}, inplace=True)
/tmp/ipykernel_2563741/138892696.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,ID,Package,CWE,CVE,Date,Disclosed_Date,Platform,Version Range,GitHub URL,Source
0,SNYK-DOTNET-MAGICKNETCORE-7856102,magick.net.core,CWE-122,CVE-2023-4863,1 Sep 2024,11 Sep 2023,NuGet,"['[,13.3.0)']",https://github.com/dlemstra/Magick.NET,Snyk
1,SNYK-DOTNET-SERILOGENRICHERSCLIENTINFO-7853254,serilog.enrichers.clientinfo,CWE-348,CVE-2024-44930,30 Aug 2024,29 Aug 2024,NuGet,"['[,2.1.0)']",https://github.com/serilog-contrib/serilog-enr...,Snyk
2,SNYK-DOTNET-RESTSHARP-7853196,restsharp,CWE-113,CVE-2024-45302,30 Aug 2024,29 Aug 2024,NuGet,"['[107.0.0-preview.1,112.0.0)']",https://github.com/restsharp/RestSharp,Snyk
3,SNYK-DOTNET-BOUNCYCASTLE-7831269,bouncycastle,CWE-835,CVE-2024-30172,26 Aug 2024,12 Apr 2024,NuGet,"['[0,]']",NaN,Snyk
4,SNYK-DOTNET-BOUNCYCASTLECRYPTOGRAPHY-7831268,bouncycastle.cryptography,CWE-835,CVE-2024-30172,26 Aug 2024,12 Apr 2024,NuGet,"['[,2.3.1)']",NaN,Snyk
...,...,...,...,...,...,...,...,...,...,...
33396,GHSA-33gv-rvgq-gpxp,BTCPayServer.Client,CWE-74,CVE-2023-0493,27 Jan 2023,NaN,NuGet,"[0,1.7.5)",https://github.com/btcpayserver/btcpayserver,Git
33397,GHSA-33gv-rvgq-gpxp,BTCPayServer.Client,CWE-76,CVE-2023-0493,27 Jan 2023,NaN,NuGet,"[0,1.7.5)",https://github.com/btcpayserver/btcpayserver,Git
33398,GHSA-34p5-jp77-fcrc,github.com/rancher/rancher,CWE-77,CVE-2022-43758,25 Jan 2023,NaN,Go,"[2.5.0,2.5.17)[2.6.0,2.6.10)[2.7.0,2.7.1)",https://github.com/rancher/rancher,Git
33399,GHSA-34p5-jp77-fcrc,github.com/rancher/rancher,CWE-78,CVE-2022-43758,25 Jan 2023,NaN,Go,"[2.5.0,2.5.17)[2.6.0,2.6.10)[2.7.0,2.7.1)",https://github.com/rancher/rancher,Git


In [4]:
group_columns = ['Package', 'CWE', 'Version Range']
check_column = 'Source'
value_to_remove = 'Git'

final_df = final_df[~((final_df[check_column] == value_to_remove) & (final_df.duplicated(subset=group_columns, keep=False)))]


# final_df = final_df[(~final_df.duplicated(subset=['CVE'], keep='first')) | (final_df['CVE'].isnull())].reset_index(drop=True)
# final_df = final_df.drop_duplicates(subset=['CVE'], keep='last').reset_index(drop=True)

final_df = final_df.drop_duplicates(subset=['ID']).reset_index(drop=True)

final_df

,ID,Package,CWE,CVE,Date,Disclosed_Date,Platform,Version Range,GitHub URL,Source
0,SNYK-DOTNET-MAGICKNETCORE-7856102,magick.net.core,CWE-122,CVE-2023-4863,1 Sep 2024,11 Sep 2023,NuGet,"['[,13.3.0)']",https://github.com/dlemstra/Magick.NET,Snyk
1,SNYK-DOTNET-SERILOGENRICHERSCLIENTINFO-7853254,serilog.enrichers.clientinfo,CWE-348,CVE-2024-44930,30 Aug 2024,29 Aug 2024,NuGet,"['[,2.1.0)']",https://github.com/serilog-contrib/serilog-enr...,Snyk
2,SNYK-DOTNET-RESTSHARP-7853196,restsharp,CWE-113,CVE-2024-45302,30 Aug 2024,29 Aug 2024,NuGet,"['[107.0.0-preview.1,112.0.0)']",https://github.com/restsharp/RestSharp,Snyk
3,SNYK-DOTNET-BOUNCYCASTLE-7831269,bouncycastle,CWE-835,CVE-2024-30172,26 Aug 2024,12 Apr 2024,NuGet,"['[0,]']",NaN,Snyk
4,SNYK-DOTNET-BOUNCYCASTLECRYPTOGRAPHY-7831268,bouncycastle.cryptography,CWE-835,CVE-2024-30172,26 Aug 2024,12 Apr 2024,NuGet,"['[,2.3.1)']",NaN,Snyk
...,...,...,...,...,...,...,...,...,...,...
28148,GHSA-2pj2-gchf-wmw7,net.lingala.zip4j:zip4j,CWE-346,CVE-2023-22899,10 Jan 2023,NaN,Maven,"[0,2.11.3)",https://github.com/srikanth-lingala/zip4j,Git
28149,GHSA-2x48-p6cq-5xcw,github.com/go-sonic/sonic,CWE-22,CVE-2022-46959,23 Jan 2023,NaN,Go,"[0,1.0.5)",https://github.com/go-sonic/sonic,Git
28150,GHSA-3244-8mff-w398,github.com/gotify/server,CWE-79,NaN,10 Jan 2023,NaN,Go,"[0,2.2.3)",https://github.com/gotify/server,Git
28151,GHSA-33gv-rvgq-gpxp,BTCPayServer.Client,CWE-74,CVE-2023-0493,27 Jan 2023,NaN,NuGet,"[0,1.7.5)",https://github.com/btcpayserver/btcpayserver,Git


In [5]:
columns_to_check = ['Package', 'CWE', 'Version Range']
nan_cv_df = final_df[final_df["CVE"].isna()]

duplicates = nan_cv_df[nan_cv_df.duplicated(subset=columns_to_check, keep=False)]
duplicates

In [8]:
final_df.to_csv(f'../data/snyk_and_advisory.csv', index=False)
# final_df = pd.read_csv('data/snyk_and_advisory.csv')